## category 10 : musk (top, base)

#### 순서
1. 향수 정보 열기
2. <카테고리  3> : top_notes에 'rose'를 포함하는 향수만 필터링
3. top_notes 리스트에의 향들 : 카테고리 3에 포함되는 향 갯수, 다음 가까운 카테고리에 포함되는 향 갯수, 그 다음...
4. 각 갯수 * 해당 가중치(1, 0.7, 0.4) ==> 'score'
5. score에 따라서 정렬 후 top 3 향수 추출
6. 저장


In [23]:
import pandas as pd
from ast import literal_eval

import warnings
warnings.filterwarnings("ignore")

### 데이터 불러오기

In [30]:
# 향수 데이터를 필요한 칼럼만 남겨 perfume_df에 불러오기
perfume_df = pd.read_csv('../data/1st treatment_perfume.csv', index_col = 0)
perfume_df = perfume_df[['pk', 'name', 'top_notes', 'heart_notes', 'base_notes']]

# perfume_df의 note들 정보가 리스트가 아닌 string으로 저장되어 있으므로 리스트로 변환
note_types = ['top_notes', 'heart_notes', 'base_notes']
for n_type in note_types:
    perfume_df[n_type] = perfume_df[n_type].apply(literal_eval)

# 각 카테고리별 향들의 정보가 들어있는 cate_note.csv불러오기
cat_notes = pd.read_csv('../data/cate_note.csv', index_col = 0)

# note에 대한 정보 (이름과  pk)을 가진 데이터 프레임
note8 = pd.read_csv('../data/concat_notes.csv')
note8 = note8[['pk', 'name']]
note8[note8['name'] == 'musk'] # musk pk값 확인

,pk,name
623,624,musk


***

### 타겟 향이 top & base에 포함되는 향수 추출

#### 1. top 필터링

In [31]:
# top_note에 타겟 향 (624)을 포함하는 향수 pk값 리스트로 반환
top_624 = []

for i in range(len(perfume_df)):
    for j in range(len(perfume_df['top_notes'][i])):
        if perfume_df['top_notes'][i][j] == 624:
            top_624.append(perfume_df['pk'][i])
print('top_notes 필터 후 : ', len(top_624))

# 반환된 리스트 값을 pk로 가지는 향수 정보를 보여주는 데이터 프레임
test = []
for i in top_624:
    test.append(perfume_df[perfume_df['pk'] == i])

df = test[0]
for i in range(1,len(test)):
    df = df.append(test[i])
df =df.reset_index(drop = True)
print('필터링된 데이터 프레임 크기 : ',len(df))

top_notes 필터 후 :  1055
필터링된 데이터 프레임 크기 :  1055


#### 2. base 필터

In [32]:
# 필터링된 데이터 프레임에서 base_notes에 타겟 향(624)을 가지는 향수의 pk값 리스트에 반환
base_624 = []

for i in range(len(df)):
    for j in range(len(df['base_notes'][i])):
        if df['base_notes'][i][j] == 624:
            base_624.append(df['pk'][i])
print('base_notes 필터 후 : ' ,len(base_624))

# 반환된 리스트의 값을 pk값으로 가지는 향수의 정보 데이터프레임으로 출력
test = []
for i in base_624:
    test.append(perfume_df[perfume_df['pk'] == i])

df_base = test[0]
for i in range(1,len(test)):
    df_base = df_base.append(test[i])
df_final = df_base.reset_index(drop = True)
print('필터링된 데이터 프레임 크기 : ', len(df_base))

base_notes 필터 후 :  5
필터링된 데이터 프레임 크기 :  5


***

#### 필터링이 된 df(df_final)에서 점수 계산 후 탑3개
* cat10 = 1, cat6 = 0.7, cat9 = 0.3 점

In [33]:
cat10 = cat_notes['note_idx'].apply(literal_eval)[10] # 10번 카테고리에 해당하는 note들의 pk값 리스트
cat6 = cat_notes['note_idx'].apply(literal_eval)[6] # 6번 카테고리에 해당하는 note들의 pk값 리스트
cat9 = cat_notes['note_idx'].apply(literal_eval)[9] # 9번 카테고리에 해당하는 note들의 pk값 리스트

In [38]:
# 타겟 카테고리에 해당하는 향을 카테고리에 따라 a 또는 b 또는 c로 변경하는 함수
def convert_note(target_type):
    for i in range(len(df_final)):
        for j in range(len(df_final[target_type][i])):
            if df_final[target_type][i][j] in cat10:
                df_final[target_type][i][j] = 'a'
            elif df_final[target_type][i][j] in cat6:
                df_final[target_type][i][j] = 'b'
            elif df_final[target_type][i][j] in cat9:
                df_final[target_type][i][j] = 'c'
            else:
                pass

In [39]:
# target_type을 *_note 열들로 지정하고 위 조건에 해당하는 값을 변경
target_type = ['top_notes', 'heart_notes', 'base_notes']
for i in target_type:
    convert_note(i)
    
df_final.head(3)

,pk,name,top_notes,heart_notes,base_notes
0,26120367,cK be,"[96, 481, 512, 562, 604, a]","[555, a, 694]","[a, c, 808, b]"
1,26145605,Sea Musk,[a],"[7, a, b]",[a]
2,26146192,Cuir Musc,"[a, 765, 940]","[a, 900]","[a, a, 649]"


In [40]:
# Import Counter from collections
from collections import Counter

# 데이터 프레임에 score열을 만들고 위에서 번환된 note값에 따라 점수 계산
# 10번 카테고리 향은 1점, 6번 카테고리 향은 0.7점, 9번 카테고리 향은 0.4점

# 점수 계산 함수
def score_cal(target_type):
    score = Counter(df_final[target_type][i])['a'] * 1 + Counter(df_final[target_type][i])['b'] * 0.7 + Counter(df_final[target_type][i])['c'] * 0.4
    return score

df_final['score'] = 0
for i in range(len(df_final)):
    t_score = score_cal('top_notes')
    m_score = score_cal('heart_notes')
    b_score = score_cal('base_notes')
    df_final['score'][i] = t_score + m_score + b_score


In [42]:
# score값에 따른 top 3
df_final.sort_values(by = 'score', ascending = False)[0:3]

,pk,name,top_notes,heart_notes,base_notes,score
0,26120367,cK be,"[96, 481, 512, 562, 604, a]","[555, a, 694]","[a, c, 808, b]",4.1
2,26146192,Cuir Musc,"[a, 765, 940]","[a, 900]","[a, a, 649]",4.0
3,26146195,Amber Musc,"[a, 785]",[a],"[a, 193, a]",4.0


In [43]:
# 해당 향수의 전체 노트의 수 = 'total_notes'
df_final['total_notes'] = None
# 해당 향수의 평균 점수
df_final['score_avg'] = 0

# 노트들의 리스트 길이를 이용하여 평균 점수 구하기
for i in range(len(df_final)):
    df_final['total_notes'][i] = len(df_final['top_notes'][i] + df_final['heart_notes'][i] + df_final['base_notes'][i])
    df_final['score_avg'][i] = df_final['score'][i]/df_final['total_notes'][i]


In [47]:
# 평균 점수로 정한 top 3향수
df_final.sort_values(by = 'score_avg', ascending = False)[0:3]

,pk,name,top_notes,heart_notes,base_notes,score,total_notes,score_avg
4,26148931,Puro,[a],"[a, 193]",[a],3.0,4,0.750000
1,26145605,Sea Musk,[a],"[7, a, b]",[a],3.7,5,0.740000
3,26146195,Amber Musc,"[a, 785]",[a],"[a, 193, a]",4.0,6,0.666667


In [49]:
df_base = pd.read_csv('../data/fill_season_catagory.csv')

# 점수 계산 후 나온 상위 3개의 향수의 pk값을 이용하여 향수 최종 데이터에서 동일 pk값을 가진 향수 정보를 추출 
result = df_final.sort_values(by = 'score_avg', ascending = False)[0:3]
result_pk = result['pk'].reset_index(drop=True)

fin = df_base[df_base['pk'] == result_pk[0]]
for i in range(1, len(result_pk)):
    fin = fin.append(df_base[df_base['pk'] == result_pk[i]])

# 결과 데이터 프레임 csv로 저장
fin.to_csv('../data/category10_result.csv')
